In [20]:
from nltk.sentiment import SentimentIntensityAnalyzer

import pandas as pd
from tqdm import tqdm
import pyarrow.feather as feather

sia = SentimentIntensityAnalyzer()

#we'll look at each the sentiment in each headline and then create a daily aggregate score.
#this means we need to create a new headline importer:

news = pd.read_csv("kaggle/input/million-headlines/abcnews-date-text.csv",nrows=1000)
news["publish_date"] = pd.to_datetime(news["publish_date"].astype(str),infer_datetime_format=True)

news["scores"]= news["headline_text"].apply((lambda x :sia.polarity_scores(x)))


news = pd.concat([news,news["scores"].apply(pd.Series)],axis=1)

news = news[[c for c in news.columns if c in {"publish_date","neg","neu","pos","compound"}]]

news = news.groupby("publish_date").mean()

news.head(30)



,neg,neu,pos,compound
publish_date,,,,
2003-02-19,0.166949,0.753631,0.079424,-0.107614
2003-02-20,0.169268,0.750292,0.080436,-0.110760
2003-02-21,0.161764,0.758120,0.080116,-0.104719
2003-02-22,0.134317,0.760603,0.105087,-0.052789
2003-02-23,0.175132,0.733257,0.091610,-0.099331
2003-02-24,0.183475,0.744425,0.072100,-0.149413


In [21]:
news.index.names = ["date"]

stockdata = feather.read_feather("kaggle/input/stocks.feather")


final_data = news.merge(stockdata,how="inner",on="date").copy()

final_data.head()

,neg,neu,pos,compound,Average
date,,,,,
2003-02-19,0.166949,0.753631,0.079424,-0.107614,-0.007923
2003-02-20,0.169268,0.750292,0.080436,-0.110760,-0.002093
2003-02-21,0.161764,0.758120,0.080116,-0.104719,0.013579
2003-02-24,0.183475,0.744425,0.072100,-0.149413,-0.007012


In [24]:
stockdata.head()

,Average
date,
2008-01-28,0.015572
2008-01-29,0.003104
2008-01-30,0.001356
2008-01-31,0.028865
2008-02-01,0.012284


In [32]:
from datetime import timedelta
stockdata["newdate"] = stockdata.index
stockdata["newdate"] = stockdata["newdate"]-timedelta(days=1)
stockdata.head()

,Average,newdate
date,,
2008-01-28,0.015572,2008-01-27
2008-01-29,0.003104,2008-01-28
2008-01-30,0.001356,2008-01-29
2008-01-31,0.028865,2008-01-30
2008-02-01,0.012284,2008-01-31


In [33]:
stockdata = stockdata.set_index("newdate")
stockdata.head()

,Average
newdate,
2008-01-27,0.015572
2008-01-28,0.003104
2008-01-29,0.001356
2008-01-30,0.028865
2008-01-31,0.012284
